In [44]:
import pandas as pd
import numpy as np
import matplotlib as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from var import COL_NAME, MONTH_CONVERT
pd.set_option("display.max_rows", 999)


In [48]:
iter_data = pd.read_csv("./data/export_data.csv", sep=";", decimal=',', encoding="latin-1", chunksize=500)
data= pd.concat([i for i in iter_data])
data.columns = map(str.lower, data.columns)
data["year_month"] = data["mois_annee_fr"].str[-4:] + data["mois_annee_fr"].str[:3].map(MONTH_CONVERT)
data = data[COL_NAME]

data = data[(data["rea_post_op"]==1) & (data['sexe']!='I')].dropna()
data ['imc'] = round(data.poids / (data.taille * 10**(-2))**2,2)
data['asa_score'] = [ i[-1]for i in data["asa"]]
data["sexe"] = data["sexe"].replace({"M":"1", "F":"0"})
data[["sexe", "age", "urgence", "asa_score"]] = data[["sexe", "age", "urgence", "asa_score"]].astype(int)

# data = data[["age", "imc", "asa_score", "urgence", "sexe", "service", "duree_rea_post_op"]]

In [50]:
len(data["year_month"].unique())

60

In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, LeaveOneOut
import pickle
res_name = []
res_val = []
nmae_score = []
for name, df in data.groupby("service"):
    print(name)
    y="duree_rea_post_op"
    ft = [i for i in df.columns if (i !=y) & (i != "service")]
    
    X = df[ft].values
    y = df[y].values
#     print(X)
    cv = LeaveOneOut()
    rfr = RandomForestRegressor()
    cv_results = cross_val_score(rfr, X =X, y =y , cv = 2, n_jobs=-1, scoring="neg_median_absolute_error")
    rfr.fit(X,y)
    
    filename = './save_model/' + name + '.sav'
    pickle.dump(rfr, open(filename, 'wb'))
    
    res_name.append(name)
    res_val.append( np.round(rfr.feature_importances_,2)*100)
    nmae_score.append(np.round(np.mean(cv_results),3))
#     print(np.mean(cv_results))
#     print(np.round(rfr.feature_importances_,2)*100)

results = pd.DataFrame(res_val, columns=ft, index=res_name)
results["NMAE_Score"] = nmae_score
    

Bloc Commun
Brûlés
CCV
CMCA
Chir Pédiatrique
Chir Tho
Gynécologie
NeuroChirurgie
NeuroRadiologie
Non définie
ORL
Obstétrique
Ophtalmologie
Spécialités Salengro
Traumatologie
Urgences


In [59]:
results[ft] = results[ft].astype(int)


In [73]:
data

,year_month,id_intervention,categorie_age_adulte,age,poids,taille,asa,urgence,sexe,lib_diagnostic,service,rea_post_op,duree_rea_post_op,imc,asa_score
62,201304,471500,4.0,59,88.0,170.0,ASA4,0,1,"Anévrisme aortique thoracique, sans mention de...",CCV,1.0,23.0,30.45,4
110,201306,486500,5.0,69,65.0,168.0,ASA3,0,1,Endocardite mitrale aiguë,CCV,1.0,7.0,23.03,3
159,201309,496700,3.0,42,73.0,174.0,ASA1,1,1,Tétraplégie flasque,NeuroChirurgie,1.0,3.0,24.11,1
183,201306,484000,3.0,41,116.0,160.0,ASA2,1,0,"Hydrocéphalie, sans précision",NeuroChirurgie,1.0,9.0,45.31,2
199,201305,480900,6.0,72,86.0,157.0,ASA3,0,0,Sténose (de la valvule) aortique (non rhumatis...,CCV,1.0,2.0,34.89,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298608,201607,693050,5.0,64,57.0,158.0,ASA3,0,0,Présence de greffe de coeur et poumon,CCV,1.0,3.0,22.83,3
298623,201703,732888,6.0,76,81.0,169.0,ASA3,0,1,Myocardiopathie ischémique,CCV,1.0,2.0,28.36,3
298730,201703,726016,3.0,43,92.0,168.0,ASA3,0,0,Bloc auriculoventriculaire complet,CCV,1.0,5.0,32.60,3
298732,201303,469989,4.0,53,50.0,178.0,ASA3,0,1,"Soin de contrôle chirurgical, sans précision",CCV,1.0,118.0,15.78,3


In [61]:
results.reset_index().to_csv("./save_model/df_result.csv")

In [72]:
pd.melt(results.reset_index(), id_vars='index', value_vars = ft).columns

Index(['index', 'variable', 'value'], dtype='object')